# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

city_data_df["Max Temp"] = (city_data_df["Max Temp"]) 


# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,-0.46,86,92,7.49,GS,1694970049
1,1,katsuura,35.1333,140.3000,26.23,88,4,5.94,JP,1694970050
2,2,yellowknife,62.4560,-114.3525,13.56,88,99,5.36,CA,1694970050
3,3,port-aux-francais,-49.3500,70.2167,5.48,97,100,13.11,TF,1694970051
4,4,west island,-12.1568,96.8225,24.99,83,100,7.72,CC,1694970051


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True,
    color = "City",
    alpha = 0.6,
    size = "Humidity",
    tiles = "OSM",
    width = 1100,
    height = 600                                                                          
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values

# My ideal weather condition criteria:
# Temperature between 23 and 30 degrees
# Humidity under 80%
# Cloudiness of 0

ideal_weather_df = city_data_df[(city_data_df["Max Temp"] >= 23) &
                                (city_data_df["Max Temp"] <= 30) &
                                (city_data_df["Humidity"] < 80) &
                                (city_data_df["Cloudiness"] == 0)]
                            

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,51,ghanzi,-21.5667,21.7833,24.12,11,0,3.26,BW,1694970052
61,61,cidreira,-30.1811,-50.2056,23.45,65,0,3.22,BR,1694970076
69,69,arraial do cabo,-22.9661,-42.0278,28.92,61,0,8.75,BR,1694970078
128,128,eastham,41.8301,-69.9739,26.44,59,0,0.89,US,1694970101
155,155,great falls,47.5002,-111.3008,26.66,21,0,7.20,US,1694970109
180,180,celestun,20.8667,-90.4000,28.08,75,0,2.19,MX,1694970115
182,182,paris,48.8534,2.3488,27.28,65,0,4.12,FR,1694969793
231,231,great bend,38.3645,-98.7648,24.46,67,0,4.12,US,1694970124
244,244,swift current,50.2834,-107.8014,26.24,16,0,5.09,CA,1694970125
301,301,westport,41.1415,-73.3579,26.38,54,0,3.09,US,1694970137


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
51,ghanzi,BW,-21.5667,21.7833,11,
61,cidreira,BR,-30.1811,-50.2056,65,
69,arraial do cabo,BR,-22.9661,-42.0278,61,
128,eastham,US,41.8301,-69.9739,59,
155,great falls,US,47.5002,-111.3008,21,
180,celestun,MX,20.8667,-90.4000,75,
182,paris,FR,48.8534,2.3488,65,
231,great bend,US,38.3645,-98.7648,67,
244,swift current,CA,50.2834,-107.8014,16,
301,westport,US,41.1415,-73.3579,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 1000
params = {
    "radius": radius,
    "type": "hotel",  
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df["Lat"]
    longitude = hotel_df["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ghanzi - nearest hotel: No hotel found
cidreira - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
eastham - nearest hotel: No hotel found
great falls - nearest hotel: No hotel found
celestun - nearest hotel: No hotel found
paris - nearest hotel: No hotel found
great bend - nearest hotel: No hotel found
swift current - nearest hotel: No hotel found
westport - nearest hotel: No hotel found
pocahontas - nearest hotel: No hotel found
mulanje - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
karatas - nearest hotel: No hotel found
nantucket - nearest hotel: No hotel found
quelimane - nearest hotel: No hotel found
north battleford - nearest hotel: No hotel found
gokwe - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
51,ghanzi,BW,-21.5667,21.7833,11,No hotel found
61,cidreira,BR,-30.1811,-50.2056,65,No hotel found
69,arraial do cabo,BR,-22.9661,-42.0278,61,No hotel found
128,eastham,US,41.8301,-69.9739,59,No hotel found
155,great falls,US,47.5002,-111.3008,21,No hotel found
180,celestun,MX,20.8667,-90.4000,75,No hotel found
182,paris,FR,48.8534,2.3488,65,No hotel found
231,great bend,US,38.3645,-98.7648,67,No hotel found
244,swift current,CA,50.2834,-107.8014,16,No hotel found
301,westport,US,41.1415,-73.3579,54,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"],
                                   width = 1000,
                                   height = 600 
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)